In [1]:
from langchain.llms import Ollama

llm = Ollama(model="solar")

In [2]:
from llama_index import download_loader

WikipediaReader = download_loader("WikipediaReader")

loader = WikipediaReader()

documents = loader.load_data(pages=['Natural Language Processing', 'Artificial Intelligence'])
print(len(documents))

2


In [3]:
from llama_index.vector_stores import DeepLakeVectorStore

my_activeloop_org_id = "tobeetaylor"
my_activeloop_dataset_name = "LlamaIndex_intro"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"

# Create an index over the documnts
vector_store = DeepLakeVectorStore(dataset_path=dataset_path, overwrite=False)

Deep Lake Dataset in hub://tobeetaylor/LlamaIndex_intro already exists, loading from the storage


In [4]:
from llama_index.storage.storage_context import StorageContext
from llama_index.service_context import ServiceContext
from llama_index import VectorStoreIndex

storage_context = StorageContext.from_defaults(
    vector_store=vector_store
)

service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model="local:BAAI/bge-base-en-v1.5"
)

index = VectorStoreIndex.from_documents(
    documents, 
    storage_context=storage_context,
    service_context=service_context
)

/Users/tobiasoberrauch/Repositories/sandbox/rag/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
config.json: 100%|██████████| 777/777 [00:00<00:00, 1.50MB/s]
model.safetensors: 100%|██████████| 438M/438M [00:05<00:00, 78.6MB/s] 
tokenizer_config.json: 100%|██████████| 366/366 [00:00<00:00, 692kB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 1.14MB/s]
tokenizer.json: 100%|██████████| 711k/711k [00:00<00:00, 1.95MB/s]
special_tokens_map.json: 100%|██████████| 125/125 [00:00<00:00, 319kB/s]


Uploading data to deeplake dataset.


100%|██████████| 25/25 [00:00<00:00, 34.35it/s]
/

Dataset(path='hub://tobeetaylor/LlamaIndex_intro', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
 embedding  embedding  (25, 768)  float32   None   
    id        text      (25, 1)     str     None   
 metadata     json      (25, 1)     str     None   
   text       text      (25, 1)     str     None   


In [6]:
query_engine = index.as_query_engine()
response = query_engine.query("What does NLP stands for?")
response.response

/Users/tobiasoberrauch/Repositories/sandbox/rag/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'NLP stands for Natural Language Processing.'

In [7]:
index.storage_context.persist()

In [10]:
# Index Storage Checks
import os.path
from llama_index import (
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index import download_loader

WikipediaReader = download_loader("WikipediaReader")
loader = WikipediaReader()

# Let's see if our index already exists in storage.
if not os.path.exists("./storage"):
    # If not, we'll load the Wikipedia data and create a new index
    documents = loader.load_data(pages=['Natural Language Processing', 'Artificial Intelligence'])
    index = VectorStoreIndex.from_documents(documents, storage_context=storage_context, service_context=service_context)
    # Index storing
    index.storage_context.persist()

else:
    # If the index already exists, we'll just load it:
    storage_context = StorageContext.from_defaults(persist_dir="./storage")
    index = load_index_from_storage(storage_context)

KeyError: 'default'